In [1]:
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain.prompts import ChatPromptTemplate

from dotenv import load_dotenv

load_dotenv()



True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [2]:
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv


load_dotenv()

llm = ChatOpenAI(
    model="gpt-4o",temperature=0.9,max_tokens=500
)

@tool
def SqlQueryWritter(user_input : str)->str:
    """
    A tool that generates SQL queries based on user input.
    """
   
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system","you are an helpfull assistent to write the SQL Queries"),
        
            ("user","{query}")
        ]
    )

    chain = prompt | llm 

    response = chain.invoke({"query":user_input})

    return response.content



@tool
def SqlQuerySummarizer(user_input : str)->str:
    """
    A tool that generates SQL queries based on user input.
    """
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system","you are an helpfull assistent , Summarize the SQL Queries"),
        
            ("user","{query}")
        ]
    )

    chain = prompt | llm 

    response = chain.invoke({"query":user_input})

    return response.content


In [3]:
tools=[SqlQueryWritter,SqlQuerySummarizer]



In [25]:
# Define the tools
#tools = [SqlQueryWritter]
from langchain.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
# Create the prompt for the agent
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "you are a helpful assistant to write queries or summarize queries"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)


In [8]:
#from langchain.agents.initialize import initialize_agent
#from langchain.agents import create_react_agent


from langchain.agents import load_tools                                         
from langchain.agents import initialize_agent
# Create the agent using the tools
#agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)
chat_history = []

agent = initialize_agent(
    tools=tools,
    llm=llm,
    prompt=prompt 
)


In [26]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
llm_withtools=llm.bind_tools(tools)
agent=(
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_withtools
    | OpenAIToolsAgentOutputParser()
)

In [27]:
from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferWindowMemory
#memory = ConversationBufferWindowMemory(memory_key="chat_history",input_key="input",output_key='output', return_messages=True, k=4)



agent_executor = AgentExecutor(agent=agent, tools=tools)
                               

In [29]:
res=agent_executor.invoke({"input":"top 20 records sql sql query"})

In [32]:
print(res["output"])

To retrieve the top 20 records from a database table using SQL, you can use different SQL syntax depending on the database you are using. Here are some examples:

### For MySQL, PostgreSQL, SQLite
```sql
SELECT *
FROM your_table_name
ORDER BY some_column
LIMIT 20;
```

### For SQL Server
```sql
SELECT TOP 20 *
FROM your_table_name
ORDER BY some_column;
```

### For Oracle
Oracle doesn't use `LIMIT` or `TOP`. Instead, you can use:

#### Using `FETCH` Clause
```sql
SELECT *
FROM your_table_name
ORDER BY some_column
FETCH FIRST 20 ROWS ONLY;
```

#### Using `ROWNUM`
```sql
SELECT *
FROM (
  SELECT *
  FROM your_table_name
  ORDER BY some_column
)
WHERE ROWNUM <= 20;
```

Ensure to replace `your_table_name` with the actual table name and `some_column` with the column you wish to order the records by. The `ORDER BY` clause is crucial to define which records are considered the "top" ones.


## Normal Method Function call

In [5]:
import json
from datetime import datetime,timedelta
from openai import OpenAI

openai=OpenAI()

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_delivery_date",
            "description": "Get the delivery date for a customer's order. Call this whenever you need to know the delivery date, for example when a customer asks 'Where is my package'",
            "parameters": {
                "type": "object",
                "properties": {
                    "date": {
                        "type": "string",
                        "description": "Delivery Date.",
                    },
                },
                "required": ["date"],
                "additionalProperties": False,
            },
        }
    }
]


def get_delivery_date(date :str):
    """Get flight information between two locations."""
    delivery_date = datetime.strptime(date, "%Y-%m-%d") + timedelta(days=3)

    return json.dumps({"date":delivery_date.strftime("%Y-%m-%d")})
    


user_prompt = "Get my phone delivary date?"


output = openai.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": user_prompt}],
    tools=tools
)



In [6]:
output.choices[0].message

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_bSJJiGCtx61mS74fiprUKWXS', function=Function(arguments='{"date":"2023-10-25"}', name='get_delivery_date'), type='function')])

In [11]:
tool_call = output.choices[0].message.tool_calls[0]

In [18]:
json.loads(tool_call.function.arguments).get("date")

'2023-10-25'

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_YuSIwSKhnSIkm05WsDtmKQew', function=Function(arguments='{"date":"2023-10-25"}', name='get_delivery_date'), type='function')])

In [52]:
origin = json.loads(output.function_call.arguments).get("loc_origin")
destination = json.loads(output.function_call.arguments).get("loc_destination")
params = json.loads(output.function_call.arguments)
type(params)

print(origin)
print(destination)
print(params)


AttributeError: 'ChatCompletion' object has no attribute 'function_call'

In [ ]:
import openai
import json

# # Set up Azure OpenAI credentials
# openai.api_type = "azure"
# openai.api_key = "<your_azure_openai_key>"
# openai.api_base = "https://<your-resource-name>.openai.azure.com/"
# openai.api_version = "2023-10-01"

# Define the two functions: LLM-based answering and Vector Search
functions = [
    {
        "name": "llm_generate_answer",
        "description": "Use LLM to attempt to answer the query based on its knowledge",
        "parameters": {
            "type": "object",
            "properties": {
                "question": {"type": "string", "description": "The user question to be answered by the LLM"}
            },
            "required": ["question"]
        }
    },
    {
        "name": "vector_search",
        "description": "Perform a vector search to retrieve relevant information from a database",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {"type": "string", "description": "The query to search in the vector database"},
                "top_k": {"type": "integer", "description": "The number of top results to retrieve", "default": 3}
            },
            "required": ["query"]
        }
    }
]

# Simulate LLM answer generation function
def llm_generate_answer(question):
    prompt = f"Answer the following question: {question}"
    
    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo",
        messages=[{"role": "user", "content": prompt}],
    )
    
    llm_answer = response['choices'][0]['message']['content']
    return llm_answer

# Simulate vector search function (e.g., from Pinecone, Weaviate, or other vector DBs)
def vector_search(query, top_k=3):
    # Normally, you'd connect to a real vector database, but we'll simulate results here
    vector_results = [
        {"content": "Specific result 1 from vector database, related to personal data."},
        {"content": "Specific result 2 from vector database."},
        {"content": "Specific result 3 providing deeper context."},
    ]
    return vector_results[:top_k]

# Main function to handle simultaneous function calling for LLM and vector search
def handle_parallel_calls(prompt):
    # Step 1: Trigger both LLM answer generation and vector search simultaneously

    # First, call the LLM-based answering function
    llm_response = openai.ChatCompletion.create(
        engine="gpt-35-turbo",
        messages=[{"role": "user", "content": prompt}],
        functions=[{
            "name": "llm_generate_answer",
            "description": "Use LLM to answer the query based on its knowledge",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {"type": "string", "description": "The user question to be answered by the LLM"}
                },
                "required": ["question"]
            }
        }],
        function_call="auto"
    )

    # Simultaneously, call the vector search function
    vector_response = openai.ChatCompletion.create(
        engine="gpt-35-turbo",
        messages=[{"role": "user", "content": f"Search vector database for: {prompt}"}],
        functions=[{
            "name": "vector_search",
            "description": "Search relevant information in the vector database",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "The query string."},
                    "top_k": {"type": "integer", "description": "Number of results to return.", "default": 3}
                },
                "required": ["query"]
            }
        }],
        function_call="auto"
    )

    # Step 2: Collect the responses
    llm_message = llm_response['choices'][0]['message']
    vector_message = vector_response['choices'][0]['message']

    llm_answer = ""
    vector_results = ""

    # Handle LLM function call
    if "function_call" in llm_message:
        function_name = llm_message['function_call']['name']
        arguments = json.loads(llm_message['function_call']['arguments'])

        if function_name == "llm_generate_answer":
            question = arguments['question']
            llm_answer = llm_generate_answer(question)

    # Handle vector search function call
    if "function_call" in vector_message:
        function_name = vector_message['function_call']['name']
        arguments = json.loads(vector_message['function_call']['arguments'])

        if function_name == "vector_search":
            query = arguments['query']
            vector_results = vector_search(query, arguments.get('top_k', 3))

    # Step 3: Combine results from both LLM and vector search
    final_response = llm_answer

    if vector_results:
        final_response += "\n\nAdditional Information from Vector Search:\n"
        for result in vector_results:
            final_response += result['content'] + "\n"

    # Print or return the combined response
    print(f"Final Combined Response:\n{final_response}")
    return final_response

# Example usage
user_input = "Tell me about my recent purchases or trends in the last quarter."
handle_parallel_calls(user_input)


In [41]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Write a haiku about recursion in programming."
        }
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Endless calls repeat,  \nEach layer peels the next one—  \nDepth in code unfolds.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)
